In [2]:
%load_ext autoreload
%autoreload 2

import torch
from mvn.utils import multiview
from collections import defaultdict

from mvn.datasets.cmu import CMUSceneDataset
from mvn.datasets.human36m import Human36MMultiViewDataset, Human36MSingleViewDataset
from torch.utils.data import DataLoader

import os
from itertools import islice
from tqdm import tqdm_notebook
import pickle


import numpy as np
import cv2
import time
import sys

from torch import nn
import torch.nn.functional as F
from torch.utils.data import DataLoader

from mvn.utils.img import image_batch_to_numpy, denormalize_image
from mvn.models.temporal import TemporalModel, TemporalModelBase
from mvn.models.basic import VolumetricTriangulationNet, VolumetricTemporalTriangulationNet
from mvn.utils.multiview import project_3d_points_to_image_plane_without_distortion
from mvn.utils.vis import draw_2d_pose

from mvn.utils import img
from mvn.utils import multiview
from mvn.utils import volumetric
from mvn.utils import op
from mvn.utils import vis
from mvn.utils import misc
from mvn.utils import cfg
import re

from time import time

from easydict import EasyDict

from mvn.datasets import cmu, kth
from mvn.datasets import utils as dataset_utils
from mvn.datasets.human36m import Human36MMultiViewDataset, Human36MSingleViewDataset

from train import setup_human36m_dataloaders, keypoints_to_features

from IPython.core.debugger import set_trace

from matplotlib import pyplot as plt
from warnings import filterwarnings
filterwarnings("ignore")

%matplotlib inline

retval = {
    'subject_names': ['S1', 'S5', 'S6', 'S7', 'S8', 'S9', 'S11'],
    'camera_names': ['54138969', '55011271', '58860488', '60457274'],
    'action_names': [
        'Directions-1', 'Directions-2',
        'Discussion-1', 'Discussion-2',
        'Eating-1', 'Eating-2',
        'Greeting-1', 'Greeting-2',
        'Phoning-1', 'Phoning-2',
        'Posing-1', 'Posing-2',
        'Purchases-1', 'Purchases-2',
        'Sitting-1', 'Sitting-2',
        'SittingDown-1', 'SittingDown-2',
        'Smoking-1', 'Smoking-2',
        'TakingPhoto-1', 'TakingPhoto-2',
        'Waiting-1', 'Waiting-2',
        'Walking-1', 'Walking-2',
        'WalkingDog-1', 'WalkingDog-2',
        'WalkingTogether-1', 'WalkingTogether-2']
}
# import nonechucks

device = torch.cuda.current_device()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
labels_path='/media/hpc2_storage/ibulygin/human36m-preprocessing/human36m-multiview-labels-GTbboxes.npy'
labels = np.load(labels_path, allow_pickle = True).item()

In [43]:
pivot_position = 'first'
n_frames = len(labels['table'])
dt = 3
dilation = 0
retain_every_n_frames_in_test=1
# Let's call an i-th element in the middle of the CONSECUTIVE sequence [i-dt//2, i, i+dt//2], with length `dt`, as pivot.
# Initially, consider all frames as pivots
pivot_mask = np.ones((n_frames,),dtype=np.bool)
# the whole time period covered with dilation
_time_period = dt + (dt-1)*(dilation)

if dt > 1:

    frame_idx =  labels['table']['frame_idx']

    # Mark positions where the new scene or action is starting
    change_mask = np.concatenate((frame_idx[:-1] > frame_idx[1:], [False]))
    
    if pivot_position == 'intermediate':
        # Shift that positions shuch that all non-pivot positions marked `True`
        for _ in range((_time_period//2)-1):
            change_mask[:-1] = change_mask[:-1] | change_mask[1:]
        for _ in range(_time_period//2):
            change_mask[1:] = change_mask[1:] | change_mask[:-1]
        change_mask[:_time_period//2] = True
        change_mask[-(_time_period//2):] = True
        
    elif pivot_position == 'first':
        # Shift that positions shuch that all non-pivot positions marked `True`
        for _ in range(_time_period-1):
            change_mask[1:] = change_mask[1:] | change_mask[:-1]
        change_mask[:_time_period-1] = True
    
    pivot_mask = ~change_mask

pivot_indxs = np.arange(n_frames)[pivot_mask][::retain_every_n_frames_in_test]
n_sequences = len(pivot_indxs)

array([     2,      3,      4, ..., 527596, 527597, 527598])